# Chapter 1: Basic Prompt Structure

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [3]:
!pip install -qU langchain_community==0.0.29

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sphinx 8.1.3 requires docutils<0.22,>=0.20, but you have docutils 0.16 which is incompatible.


In [4]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [5]:
import os
from utils import get_bedrock_client, get_haiku_config, get_sonnet_config
from langchain_community.chat_models import BedrockChat  # Changed this import

In [6]:
bedrock_client = get_bedrock_client()
sonnet_config = get_sonnet_config()  # Using Sonnet config

# Create BedrockChat instance for CrewAI
llm = BedrockChat(
    client=bedrock_client,
    model_id="anthropic.claude-3-sonnet-20240229-v1:0",  # Updated model ID
    streaming=True,
    model_kwargs={
        "temperature": sonnet_config["temperature"],
        "top_p": sonnet_config["top_p"],
        "max_tokens": sonnet_config["max_tokens"],
        "anthropic_version": "bedrock-2023-05-31"
    }
)

# Set model name in environment if needed
os.environ["MODEL_NAME"] = "anthropic.claude-3-sonnet-20240229-v1:0"

---

## Lesson

Amazon Bedrock offers three APIs that can be used with Anthropic Claude models, the legacy [Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html) , the [Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html) and the current [Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). For this tutorial, we will be exclusively using the Converse API.

At minimum, a call to Claude using the Converse API requires the following parameters:
- `modelId`: the [API model name](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns) of the model that you intend to call

- `messages`: an array of input messages. Claude 3 models are trained to operate on alternating `user` and `assistant` conversational turns. When creating a new `Message`, you specify the prior conversational turns with the messages parameter, and the model then generates the next `Message` in the conversation.
  - Each input message must be an object with a `role` and `content`. You can specify a single `user`-role message, or you can include multiple `user` and `assistant` messages (they must alternate, if so). **The first message must always use the `user` role.**
  
  You store the content for the message in the `content` field of a [(ContentBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ContentBlock.html). Specify text in the `text` field, or if supported by the model, you can also pass the raw bytes for an image in the `image` field of an [(ImageBlock)](https://docs.aws.amazon.com/bedrock/latest/APIReference/API_runtime_ImageBlock.html). The other fields in ContentBlock are for [tool use](https://docs.aws.amazon.com/bedrock/latest/userguide/tool-use.html).

There are also optional parameters, such as:
- `system`: the system prompt - more on this below.
  
- `temperature`: the degree of variability in Claude's response. For these lessons and exercises, we have set `temperature` to 0.

- `max_tokens`: the maximum number of tokens to generate before stopping. Note that Claude may stop before reaching this maximum. This parameter only specifies the absolute maximum number of tokens to generate. Furthermore, this is a *hard* stop, meaning that it may cause Claude to stop generating mid-word or mid-sentence.

For a complete list of all API parameters, visit our [API documentation](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html).

### Examples

Let's take a look at how Claude responds to some correctly-formatted prompts. For each of the following cells, run the cell (`shift+enter`), and Claude's response will appear below the block.

In [7]:
from botocore.exceptions import ClientError  # Import ClientError for exception handling

def get_completion(prompt, system_prompt=None):
    """
    Helper function to get a completion from the Bedrock client.
    
    Args:
        prompt (str): The user prompt to provide to the model.
        system_prompt (str, optional): An optional system prompt to set context for the model.

    Returns:
        str: The generated response from the model.
    """
    # Use existing configurations for inference and model parameters
    inference_config = {
        "temperature": 0.0,  # Minimal randomness
        "maxTokens": 200  # Maximum token limit
    }
    additional_model_fields = {
        "top_p": 1  # Full probability distribution
    }

    # Build the message structure, including system prompt in the user prompt context
    messages = [{"role": "user", "content": [{"text": f"{system_prompt}\n{prompt}" if system_prompt else prompt}]}]

    # Set up the Bedrock client request
    converse_api_params = {
        "modelId": sonnet_config["model"],  # Use the model ID from sonnet_config
        "messages": messages,
        "inferenceConfig": inference_config,
        "additionalModelRequestFields": additional_model_fields
    }

    # Make the API call
    try:
        response = bedrock_client.converse(**converse_api_params)
        # Extract and return the generated content
        return response['output']['message']['content'][0]['text']

    except ClientError as err:
        print(f"Client error occurred: {err.response['Error']['Message']}")
        return None

    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
        return None


In [8]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

Hello! As an AI language model, I don't have feelings or emotions, but I'm operating properly and ready to assist you with any questions or tasks you may have. How can I help you today?


In [9]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

The ocean does not have a single, uniform color. The color of the ocean can vary greatly depending on several factors:

1. Water depth: The deeper the water, the darker the blue color appears. This is because the longer wavelengths of light (reds and oranges) are absorbed more quickly by the water, leaving the shorter wavelengths of blue to penetrate deeper.

2. Sediments and particles: Areas with a lot of sediments or suspended particles in the water can appear more greenish, brownish, or even reddish, depending on the composition of the particles.

3. Phytoplankton: The presence of microscopic plant-like organisms called phytoplankton can give the ocean a greenish tint, especially during algal blooms.

4. Angle of sunlight: The angle at which sunlight hits the water surface can affect the perceived color. When the sun is directly overhead, the


In [10]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

Celine Dion was born on March 30, 1968.

So she was born in the year 1968.


Now let's take a look at some prompts that do not include the correct Converse API formatting. For these malformatted prompts, the Converse API returns an error.

First, we have an example of a Converse API call that lacks `role` and `content` fields in the `messages` array.

In [16]:
# Define the conversation messages with alternating roles
messages = [
    {"role": "user", "content": [{"text": "What year was Celine Dion born in?"}]},
    {"role": "assistant", "content": [{"text": "Celine Dion was born on March 30, 1968. Do you need more details about her life or career?"}]},
    {"role": "user", "content": [{"text": "Also, can you tell me some other facts about her?"}]}
]

# Correct inference configuration
inference_config = {
    "temperature": sonnet_config["temperature"],
    "topP": sonnet_config["top_p"],  # Corrected key
    "maxTokens": sonnet_config["max_tokens"]  # Ensure this matches your configuration
}

# Construct the request parameters
converse_api_params = {
    "modelId": "anthropic.claude-3-sonnet-20240229-v1:0",  # Model ID already set in environment
    "messages": messages,
    "inferenceConfig": inference_config
}

# Send the request to Bedrock
try:
    response = bedrock_client.converse(**converse_api_params)
    
    # Extract and print the response
    print(response['output']['message']['content'][0]['text'])

except ClientError as err:
    print(f"Client error occurred: {err.response['Error']['Message']}")

except Exception as e:
    print(f"An unexpected error occurred: {str(e)}")


Here are some additional facts about Celine Dion:

- She was born in Charlemagne, Quebec, Canada to a large family of 14 children.

- She first gained international acclaim after winning the 1988 Eurovision Song Contest representing Switzerland.

- Her English-language debut album, Unison, was released in 1990 and established her as a viable pop artist in North America.

- Some of her biggest hit songs include "The Power of Love," "Because You Loved Me," "My Heart Will Go On" from the movie Titanic, and "That's the Way It Is."

- She has won 5 Grammy Awards, including Album of the Year and Record of the Year.

- From 2003 to 2007, she had a very successful residency show called A New Day... at Caesars Palace in Las Vegas.

- She has been married to her manager, Rene Angelil, since 1994 until his death in 2016.

- With record sales of over 200 million worldwide, she remains one of the best-selling artists of all time.

- She has actively been involved in philanthropic pursuits, starting